# TODO:
- Create model and layers
- Review the differences in layers (mostly looking at the first numerical argument in the layer creation)
- Use a subset of the train/test data to create MVP

In [1]:
import sklearn as sk
from keras import layers, models, optimizers
from keras.preprocessing.image import ImageDataGenerator
import os

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
## Setup constants
IMAGE_SHAPE=(256, 256, 3)

IMAGE_TEST_DIR = '../data/hands_test'
IMAGE_TRAIN_DIR = '../data/hands_train'
IMAGE_TEST_FILENAMES = os.listdir(IMAGE_TEST_DIR)
IMAGE_TRAIN_FILENAMES = os.listdir(IMAGE_TRAIN_DIR)

In [3]:
def create_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu',
                            input_shape=IMAGE_SHAPE))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.RMSprop(lr=1e-4),
                  metrics=['acc'])
    return model

create_model()

In [4]:
def create_data_generator():
    # All images will be rescaled by 1./255
    train_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
            # This is the target directory
            IMAGE_TRAIN_DIR,
            target_size=[IMAGE_SHAPE[0], IMAGE_SHAPE[1]],
            batch_size=20,
            # Since we use binary_crossentropy loss, we need binary labels
            class_mode='binary')

    validation_generator = test_datagen.flow_from_directory(
            IMAGE_TEST_DIR,
            target_size=[IMAGE_SHAPE[0], IMAGE_SHAPE[1]],
            batch_size=20,
            class_mode='binary')
    
    return [train_generator, validation_generator]

create_data_generator()

Found 18661 images belonging to 2 classes.
Found 3299 images belonging to 2 classes.


In [5]:
model = create_model()
train_generator, validation_generator = create_data_generator()

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=50)

model.save('../data/posters_model.h5')

Found 18661 images belonging to 2 classes.
Found 3299 images belonging to 2 classes.
Epoch 1/10
100/100 [==============================] - 299s 3s/step - loss: 0.1626 - acc: 0.9520 - val_loss: 0.0260 - val_acc: 0.9915
Epoch 2/10
100/100 [==============================] - 297s 3s/step - loss: 0.0340 - acc: 0.9930 - val_loss: 0.0087 - val_acc: 0.9982
Epoch 3/10
100/100 [==============================] - 297s 3s/step - loss: 5.9820 - acc: 0.6245 - val_loss: 8.0509 - val_acc: 0.4950
Epoch 4/10
100/100 [==============================] - 296s 3s/step - loss: 7.8915 - acc: 0.5050 - val_loss: 8.0509 - val_acc: 0.4950
Epoch 5/10
100/100 [==============================] - 295s 3s/step - loss: 7.8835 - acc: 0.5055 - val_loss: 8.0509 - val_acc: 0.4950
Epoch 6/10
100/100 [==============================] - 301s 3s/step - loss: 7.8118 - acc: 0.5100 - val_loss: 8.0509 - val_acc: 0.4950
Epoch 7/10
100/100 [==============================] - 297s 3s/step - loss: 7.9792 - acc: 0.4995 - val_loss: 8.0509 - 

KeyboardInterrupt: 